### Load Pdf files

### PyPDFLoader

In [2]:
from langchain_community.document_loaders import PyPDFLoader

try:
    pypdf_loader = PyPDFLoader(file_path="data/pdf/attention.pdf")
    pypdf_docs = pypdf_loader.load()
    print(f"Loaded {len(pypdf_docs)} pages")
    print(f"Page 1 : {pypdf_docs[0].page_content[:100]}")
    print(f"Metadata: {pypdf_docs[0].metadata}")
except Exception as e:
    print(f"Error : {e}")

Loaded 15 pages
Page 1 : Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and
Metadata: {'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'author': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'data/pdf/attention.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1'}


### PyMuPDFLoader

In [6]:
from langchain_community.document_loaders import PyMuPDFLoader

try:
    pymupdf_loader = PyMuPDFLoader(file_path="data/pdf/attention.pdf")
    pymupdf_docs = pymupdf_loader.load()
    print(f"Loaded {len(pymupdf_docs)} pages")
    print(f"Page 1: {pymupdf_docs[0].page_content[:100]}")
    print(f"Metadata: {pymupdf_docs[0].metadata}")
except Exception as e:
    print(f"Error: {e}")

Loaded 15 pages
Page 1: Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and
Metadata: {'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'source': 'data/pdf/attention.pdf', 'file_path': 'data/pdf/attention.pdf', 'total_pages': 15, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'trapped': '', 'modDate': 'D:20240410211143Z', 'creationDate': 'D:20240410211143Z', 'page': 0}
